## *Special Token Operations
#### 04.4 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2023-02-09 (started 2021-11-28)


### Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the credentials

In [1]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 5 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json

In [3]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['api_token'])
algod_client.status()['last-round']

39022603

## Modifications of an ASA
Most properties of an ASA can **never** be modified
* Name and unit name
* Decimals
* Total supply

Using the special roles, it is however possible to ...
* Freeze/unfreeze ASA assets of a specific address
* Claim ASA back from a specific address
* Destroy an ASA
* Change the special roles


### First cretate Tempcoin
To study the whole life of an ASA, we create a new one, called `Tempcoin`, with the symbol `TEMP` (for temporary), with the following roles:

* **Manager** = Alice
* **Reserve** = Bob
* **Freeze** = Charlie
* **Clawback** = Dina (this will change later to Alice)

In [4]:
# Step 1: Prepare
sp = algod_client.suggested_params()
token_supply = 100                               # Token supply
token_decimals =  1                              # How many digits after the comma?
token_total = token_supply * 10**token_decimals  # Specify SMALLER unit ("cents")

token_name  = "Peters Tempcoin"                  # <----- YOUR NAME HERE
token_url   = "en.wikipedia.org/wiki/Temporary"  # <----- CHANGE if you want to
token_unitname = "TEMP"

# Step 2: Asset creation tansaction
txn = AssetConfigTxn(
    sender=MyAlgo['public'],                   # Creator of the ASA
    sp=sp,                                     # Network parameters
    total=token_total,                         # Token supply in SMALL unit
    decimals=token_decimals,
    default_frozen=False,                      
    unit_name=token_unitname,                       
    asset_name=token_name,
    url=token_url,
    manager=Alice['public'],                   # Special roles
    reserve=Bob['public'],
    freeze=Charlie['public'],
    clawback=Dina['public'], 
)

# Step 3+4: Sign and send
stxn = txn.sign(MyAlgo['private'])             # Sign
txid = algod_client.send_transaction(stxn)     # Send
print(txid)

# Step 5: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

T245FXXF43KDCHXTOUB2FMLLOVDBPFAJIBPIU5WQIJ6BMY4SN2ZQ
Current round is  39022604.
Waiting for round 39022604 to finish.
Waiting for round 39022605 to finish.
Transaction T245FXXF43KDCHXTOUB2FMLLOVDBPFAJIBPIU5WQIJ6BMY4SN2ZQ confirmed in round 39022606.


In [5]:
# Step 6: obtain asset_id
TEMP_id = txinfo['asset-index']
print(TEMP_id)

641964302


In [6]:
asset_holdings_df(algod_client,MyAlgo['public'])

,amount,unit,asset-id,name,decimals
0,37.943357,ALGO,0,Algorand,6
1,967.000000,PWSC,592359981,Peters WSC coin,2
2,1.000000,LUG1,592373389,Lugano NFT,0
3,1000.000000,WSC,640596559,Peters WSC coin,2
4,1000.000000,WSC,641958950,Peters WSC coin,2
5,1000.000000,WSC,641960001,Peters WSC coin,2
6,75.000000,TEMP,641963975,Peters Tempcoin,1
7,100.000000,TEMP,641964302,Peters Tempcoin,1


## The Reserve address

##### Step 1: Check in Pera Explorer
* Total supply
* Circulating supply

In [7]:
print(cred['explore_test']+'asset/{}'.format(TEMP_id))

https://testnet.explorer.perawallet.app/asset/641964302


##### Step 2: Transfer 50% to the reserve address
* So that they are shown as "not yet minted"

##### Step 2a: Bob opts in
* Even though Bob is the reserve, he has to opt in

In [8]:
# Step 1: prepare and create TX
sp = algod_client.suggested_params()
amt = int(0)                              # <------ opt-in = 0 transaction

txn = AssetTransferTxn(
    sender=Bob['public'],                 # <------- From Bo ...
    sp=sp,
    receiver=Bob['public'],               # <------- ... to Bob
    amt=amt,
    index=TEMP_id)                        # <----- Correct index TEMP_id

# Step 2+3: sign and send
stxn = txn.sign(Bob['private'])           # <----- Signed by Bob
txid = algod_client.send_transaction(stxn)
print(txid)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

2GOESKRH5RXLIFF2FKIPEPVJCOZPJ74EHEF4PQG35EMAXUF5B2MQ
Current round is  39022607.
Waiting for round 39022607 to finish.
Waiting for round 39022608 to finish.
Transaction 2GOESKRH5RXLIFF2FKIPEPVJCOZPJ74EHEF4PQG35EMAXUF5B2MQ confirmed in round 39022609.


##### Step 2b: Creator (=MyAlgo) transfers to the reserve (= Bob)

In [9]:
# Step 1: prepare and create TX
sp = algod_client.suggested_params()
amt = int(token_total / 2)                # <------- half of the holdings
txn = AssetTransferTxn(
    sender=MyAlgo['public'],
    sp=sp,
    receiver=Bob['public'],               # <------- Bob is the reserve
    amt=amt,
    index=TEMP_id)                        # <------- Correct index TEMP_id

# Step 2+3: sign and send
stxn = txn.sign(MyAlgo['private'])         # <----- Signed by the creator of the asset
txid = algod_client.send_transaction(stxn)
print(txid)

# Step 4: wait for confirmation
wait_for_confirmation(algod_client, txid)

CPBGHXABCXF62KONU7WFRM56B3J72GDXCPHHWKIAS6G4VO6ECMIA
Current round is  39022609.
Waiting for round 39022609 to finish.
Waiting for round 39022610 to finish.
Transaction CPBGHXABCXF62KONU7WFRM56B3J72GDXCPHHWKIAS6G4VO6ECMIA confirmed in round 39022611.


{'confirmed-round': 39022611,
 'pool-error': '',
 'txn': {'sig': 'W0AGq7qhG0xQ2dqFKgYMoc9//nlZEwosY622DO33Qaz85acZU0/jw0ld9aA+IBxe3U7N3NWHxs6W7ML1aN6xCQ==',
  'txn': {'aamt': 500,
   'arcv': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU',
   'fee': 1000,
   'fv': 39022609,
   'gen': 'testnet-v1.0',
   'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
   'lv': 39023609,
   'snd': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI',
   'type': 'axfer',
   'xaid': 641964302}}}

In [10]:
asset_holdings_df(algod_client,Bob['public'])

,amount,unit,asset-id,name,decimals
0,22.580488,ALGO,0,Algorand,6
1,2.097569,USDC,10458941,USDC,6
2,8.100000,PWSC,592359981,Peters WSC coin,2
3,25.000000,TEMP,641963975,Peters Tempcoin,1
4,50.000000,TEMP,641964302,Peters Tempcoin,1


##### Step 3: Check "Circulating Supply" in Pera Explorer

In [11]:
print(cred['explore_test']+'asset/{}'.format(TEMP_id))

https://testnet.explorer.perawallet.app/asset/641964302


## Clawback a transaction
* Normally, only the owner of a token could authorize a transfer of his tokens ... except for the clawback address `Dina`.
* She claws back 25 tokens from Bob by specifying Bob as `revocation_target`
* The clawed back coins are sent to the creator
* No opt-in necessary for clawback, because she does not receive any coins

##### Step 1: Clawback transaction
* using `revocation_target`

In [12]:
sp = algod_client.suggested_params()

# Step 1: Create clawback transaction
txn = AssetTransferTxn(
    sender=Dina['public'],                  # <---- Clawback = Dina is sender of transaction
    sp=sp,
    receiver=MyAlgo["public"],              # <---- Money goes back to MyAlgo
    amt=25*10,                              # <---- Amount in SMALL units
    index=TEMP_id,
    revocation_target=Bob['public']         # <---- Take the money out of Bob's account
    )

# Step 2: sign by clawback
stxn = txn.sign(Dina['private'])            # <---- Signed by Clawback = Dina

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: Wait for the transaction to be confirmed
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  39022612.
Waiting for round 39022612 to finish.
Waiting for round 39022613 to finish.
Transaction 6LYR6EXHCNNGWAA7QAVC7LPYHVZHS6D5A2Y7JZFMRLYX7UCZUCEA confirmed in round 39022614.


##### Step 2: Check Bob's holdings
* He now has 25 fewer coins

In [13]:
# Check Bob's asset holdings
asset_holdings_df(algod_client,Bob['public'])

,amount,unit,asset-id,name,decimals
0,22.580488,ALGO,0,Algorand,6
1,2.097569,USDC,10458941,USDC,6
2,8.100000,PWSC,592359981,Peters WSC coin,2
3,25.000000,TEMP,641963975,Peters Tempcoin,1
4,25.000000,TEMP,641964302,Peters Tempcoin,1


##### Step 3: Where did the 25 coins go?
* They always go to the creator (not the the clawback)

In [14]:
asset_holdings_df(algod_client,MyAlgo['public'])

,amount,unit,asset-id,name,decimals
0,37.942357,ALGO,0,Algorand,6
1,967.000000,PWSC,592359981,Peters WSC coin,2
2,1.000000,LUG1,592373389,Lugano NFT,0
3,1000.000000,WSC,640596559,Peters WSC coin,2
4,1000.000000,WSC,641958950,Peters WSC coin,2
5,1000.000000,WSC,641960001,Peters WSC coin,2
6,75.000000,TEMP,641963975,Peters Tempcoin,1
7,75.000000,TEMP,641964302,Peters Tempcoin,1


## Reconfigure the asset
* A reconfiguration is changing any of the four special roles
* The **Manager** (Alice) can reconfigure the asset. 
* In a reconfiguration, all four roles must again be specified
* No opt-in is necessary for reconfigure, because no coins are receved

**Example**
* Alice removes the clawback role from Dina and assigns it to herself
* Similar to `AssetConfigTxn` as when creating an asset

In [15]:
sp = algod_client.suggested_params()

# Step 1: Create Asset reconfiguration tansaction
txn = AssetConfigTxn(
    sender=Alice['public'],                    # Manager
    sp=sp,                                     # Network parameters
    index=TEMP_id,                             # <------------ MUST specify the asset ID
    manager=Alice['public'],                   # Old role, repeated
    reserve=Bob['public'],                     # Old role, repeated
    freeze=Charlie['public'],                  # Old role, repeated
    clawback=Alice['public'],                  # <-------- New responsibiliy
)

# Step 2+3: Sign and send
stxn = txn.sign(Alice['private'])              # <---- Signed by the manager!
txid = algod_client.send_transaction(stxn)     # Send
print(txid)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

UPRJZJAMZT6O4ALXJTAI5P66EMGKNBTUB6F5AJTFJ6FFPIYESBDA
Current round is  39022616.
Waiting for round 39022616 to finish.
Waiting for round 39022617 to finish.
Transaction UPRJZJAMZT6O4ALXJTAI5P66EMGKNBTUB6F5AJTFJ6FFPIYESBDA confirmed in round 39022618.


In [16]:
# Open in Pera Explorer: manager and clawback are now the same
print(cred['explore_test']+'asset/{}'.format(TEMP_id))

https://testnet.explorer.perawallet.app/asset/641964302


**TRY THIS** If Dina tries to clawback the other half of Bob's holdings, it will not work

## Freeze assets

#### Charlie freezes Bob's assets
* The freeze role (= Charlie) freezes the asset holdings of Bob.
* New transaction type `AssetFreezeTxn`
* No opt-in necessary for freezing

In [17]:
sp = algod_client.suggested_params()

# Step 1: Asset freeze tansaction
txn = AssetFreezeTxn(
    sender=Charlie['public'],                # <---- Must be initiated by freeze address
    sp=sp,
    index=TEMP_id,
    target=Bob["public"],                    # <---- This is the address that we want to freeze  
    new_freeze_state=True   
    )

# Step 2+3: Sign and send
stxn = txn.sign(Charlie['private'])          # <---- Must be signed by the freeze role(=Charlie)
txid = algod_client.send_transaction(stxn)
print(txid)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

7TEVBG6NCQWJNLI4Q3AGAGV55T5APYESJJPP7MRSJZNPQNBE4QOQ
Current round is  39022618.
Waiting for round 39022618 to finish.
Waiting for round 39022619 to finish.
Transaction 7TEVBG6NCQWJNLI4Q3AGAGV55T5APYESJJPP7MRSJZNPQNBE4QOQ confirmed in round 39022620.


#### Bob wants to spend some TEMP token
* Bob tries to send some TEMP token to Alice, but he cannot do it
* Only his TEMP token are frozen

In [18]:
# Step 1: prepare and create TX
sp = algod_client.suggested_params()
amt = int(5 * 10)    

txn = AssetTransferTxn(
    sender=Bob['public'],                 # <------- Bob wants to send ...
    sp=sp,
    receiver=Alice['public'],             # <------- ... to Alice  
    amt=amt,
    index=TEMP_id)                        # <----- asset_id for TEMP token

# Step 2+3: sign and send
stxn = txn.sign(Bob['private'])           # <----- Signed by Bob

try:
    txid = algod_client.send_transaction(stxn)
except algosdk.error.AlgodHTTPError as err:
    # print entire error message
    print(err)
    if ("frozen" in str(err)):                # check for specific type of error
        print("Asset is frozen")         
    txid = None
    
# Step 4: Wait for confirmation
# There is no step 4 here, because we already obtain an error on step 3

TransactionPool.Remember: transaction FKV6IASB7XVA7NUSHQED4UFAEVW2L5QLT4YJXBF44JYSNNLWQFNA: asset 641964302 frozen in BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU
Asset is frozen


#### Exercise: unfreezeing Bobs tokens
* How can we unfreeze Bobs tokens? 
* The operation is similar to freezing, except for ...

In [19]:
# Your Python code goes here ...
# Step 1: Prepare
sp = algod_client.suggested_params()

# Step 2: Asset freeze tansaction
txn = AssetFreezeTxn(
    sender=Charlie['public'],                # <---- Must be initiated by freeze address
    sp=sp,
    index=TEMP_id,
    target=Bob["public"],                    # <---- This is the address that we want to freeze  
    new_freeze_state=False   
    )

# Step 3: Sign and send
stxn = txn.sign(Charlie['private'])          # <---- Must be signed by the freeze role(=Charlie)
txid = algod_client.send_transaction(stxn)
print(txid)

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)





A5PFKFDQH5TLHQCQKAD7G7MYARTUG3ENJOETZ7SH52OTHJ3KEETQ
Current round is  39022620.
Waiting for round 39022620 to finish.
Waiting for round 39022621 to finish.
Transaction A5PFKFDQH5TLHQCQKAD7G7MYARTUG3ENJOETZ7SH52OTHJ3KEETQ confirmed in round 39022622.


## Destroy asset
* An asset can only be destroyed, if all coins are back in the creator's account (MyAlgo). 
* Then the manager (Alice) can destroy the asset.

#### Return all TEMP coins to MyAlgo and opt out
* Use `AssetTransferTxn` to send back coins
* Addtionally we have to opt out, using `close_assets_to`
    * *Closing* means sending the remaining balance to a specific address

In [20]:
# Step 1: prepare and create TX
sp = algod_client.suggested_params()
amt = int(25 * 10)    

txn = AssetTransferTxn(
    sender=Bob['public'],                
    sp=sp,
    receiver=MyAlgo['public'],            
    amt=amt,
    index=TEMP_id,                        
    close_assets_to=MyAlgo['public']      # <------- Opt out of the asset 
    )                       

# Step 2+3: sign and send
stxn = txn.sign(Bob['private'])           # <----- Signed by Bob
txid = algod_client.send_transaction(stxn)
    
# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  39022622.
Waiting for round 39022622 to finish.
Waiting for round 39022623 to finish.
Transaction JBAB2WLVBFZPNRPUH34KVIWXBAFTQEHLZ2CNOQT7H3BG56V3BEUQ confirmed in round 39022624.


In [21]:
# Open in Pera Explorer to check holders
print(cred['explore_test']+'asset/{}'.format(TEMP_id))

https://testnet.explorer.perawallet.app/asset/641964302


### Destroying the asset
* Use a special `AssetConfigTxn` transaction that has no special roles

In [22]:
# Step 1: prepare
sp = algod_client.suggested_params()

# Step 2: Create special AssetConfigTxn to destroy
txn = AssetConfigTxn(
    sender=Alice['public'],              # Manager must be sender
    sp=sp,
    index=TEMP_id,
    total = None,                        # <---- To destroy, set total to "None"
    strict_empty_address_check=False     # include this option in destroy ops
    )

#Step 2: Send 
stxn = txn.sign(Alice['private'])

# Step 3: Send the transaction and check for errors
try:
    txid = algod_client.send_transaction(stxn)
except algosdk.error.AlgodHTTPError as err:
    print(err)                                              # print entire error message
    if ("cannot destroy asset" in str(err)):                # check for specific type of error
        print("Cannot destroy asset {}, not holding all tokens.".format(TEMP_id))  
    if ("does not exist or has been deleted" in str(err)): 
        print("Cannot destroy asset. It does not exist (any more).")    

In [23]:
# Step 4: Wait for the transaction to be confirmed
wait_for_confirmation(algod_client, txid)

Current round is  39022624.
Waiting for round 39022624 to finish.
Waiting for round 39022625 to finish.
Transaction ORVXYZHT6VRY2LJQQLHSWZI7NVA7VPOX5PE4GN636NQD5XBDYMDQ confirmed in round 39022626.


{'confirmed-round': 39022626,
 'pool-error': '',
 'txn': {'sig': 'XTKsFnlHeFnhXccYUcZYz/E7A9mHAQBohx3Wh+Iasz5gS/tHhU/EM062hH5Zsd8JU2e7XBGukL38o8WdXhZJBw==',
  'txn': {'caid': 641964302,
   'fee': 1000,
   'fv': 39022624,
   'gen': 'testnet-v1.0',
   'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
   'lv': 39023624,
   'snd': 'ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU',
   'type': 'acfg'}}}

In [24]:
# Now check the "assets created" for that account.
account_info = algod_client.account_info(MyAlgo['public'])

In [25]:
account_info['created-assets']

[{'index': 592359981,
  'params': {'clawback': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU',
   'creator': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI',
   'decimals': 2,
   'default-frozen': False,
   'freeze': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU',
   'manager': 'ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU',
   'name': 'Peters WSC coin',
   'name-b64': 'UGV0ZXJzIFdTQyBjb2lu',
   'reserve': 'ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU',
   'total': 100000,
   'unit-name': 'PWSC',
   'unit-name-b64': 'UFdTQw==',
   'url': 'www.usi.ch/wsc',
   'url-b64': 'd3d3LnVzaS5jaC93c2M='}},
 {'index': 592373389,
  'params': {'clawback': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI',
   'creator': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI',
   'decimals': 0,
   'default-frozen': False,
   'freeze': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI',
   'manager': 'WSC24MVU

#### EXERCISE
* Check the holdings of MyAlgo, Alice and Bob. 
* Check the asset-id on Pera Explorer.
* Is the asset really gone from the blockchain? discuss

## Things that don't work

* Holding an ASA if you do not hold sufficient ALGOs
    * Minimum 0.1 ALGO per account plus 0.1 ALGO per ASA
* Destroying an ASA if the creator does not hold all coins